In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D, LeakyReLU
from tensorflow.keras.layers import  Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model, to_categorical
import glob
import random
import cv2
from random import shuffle
import voxel
from sklearn.preprocessing import MinMaxScaler

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def dataload():
    img = np.load("imgs.npy")
    mask = np.load("masks.npy")


    img = np.expand_dims(img, axis=-1)
    mask = np.expand_dims(mask, axis=-1) 
#     mask = to_categorical(mask) 
    
    return img, mask 

def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

# def minmaxScaler(data):

# def dice_coef(y_true, y_pred, smooth=1):
#     y_pred = K.argmax(y_pred, axis=-1)
# #     y_true = y_true[:,:,:,0]

#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     y_true_f = K.cast(y_true_f, 'float32')
#     y_pred_f = K.cast(y_pred_f, 'float32')

#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels=3):
    dice = 0

    for index in range(numLabels):
        if index == 0: continue
        dice += dice_coef_each(y_true, y_pred, index)
        
    return dice / numLabels # taking average

def dice_coef_each(y_true, y_pred, label, smooth=1):
    y_true = K.cast(K.equal(y_true, label), 'float32')
    y_pred = K.cast(K.equal(K.argmax(y_pred, axis=-1), label), 'float32')
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_0(y_true, y_pred):
    return dice_coef_each(y_true, y_pred, 0)

def dice_coef_1(y_true, y_pred):
    return dice_coef_each(y_true, y_pred, 1)

def dice_coef_2(y_true, y_pred):
    return dice_coef_each(y_true, y_pred, 2)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_pred = K.argmax(y_pred, axis=-1)
    y_true = y_true[:,:,:,1]
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.cast(y_true_f, 'float32')
    y_pred_f = K.cast(y_pred_f, 'float32')
    
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return tf.math.exp(1  - score) - 1.0
    # return 1. - score

def bce_dice_loss(y_true, y_pred):
    return categorical_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def build_unet(sz=(512,512,1)):
    x = Input(sz)
    inputs = x
  
    #down sampling 
    f = 8
    layers = []
  
    for i in range(0, 6):
        # kernel_initializer='he_norm' kernel의 값을 맞춰줄 수 있음. (초기화 설정) he_norm 앞 레이어의 평균과 표준편차를 맞춰서 정규화를 해준다.
        x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
        # bias는 BatchNormalization에서 조절
        x = Conv2D(f, 3, activation='relu', padding='same', use_bias=False, kernel_initializer='he_normal') (x)
        # BatchNormalization
        x = BatchNormalization()(x)
        x = Dropout(0.25)(x)
        layers.append(x)
        x = MaxPooling2D() (x)
        f = f*2
        ff2 = 64 
    
    #bottleneck 
    j = len(layers) - 1
    x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
    x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
    x = Dropout(0.25)(x)
    x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
    x = Concatenate(axis=3)([x, layers[j]])
    j = j -1 
  
    #upsampling 
    for i in range(0, 5):
        ff2 = ff2//2
        f = f // 2 
        x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
        x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
        x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same', use_bias=False) (x)
        # BatchNormalization
        x = BatchNormalization()(x)
        x = Dropout(0.25)(x)
        x = Concatenate(axis=3)([x, layers[j]])
        j = j -1 
    
    #classification 
    x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
    x = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='he_normal') (x)
    x = Dropout(0.25)(x)
    outputs = Conv2D(3, 1, activation='softmax') (x)
    
    #model creation 
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', 
                  metrics = [dice_coef_0, dice_coef_1, dice_coef_2])
  
    return model

1 Physical GPUs, 1 Logical GPUs


In [2]:
if __name__ == '__main__':
    import os
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    # source, target = data_convert()

    source, target = dataload()
    print(source.shape, target.shape)
    
    model = build_unet()
    model.summary()

    model.fit(source, target, epochs=100, validation_split=0.2, batch_size=8)

(3890, 512, 512, 1) (3890, 512, 512, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 8)  80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 8)  576         conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 8)  32          conv2d_1[0][0]                   
______________________________________________________

Epoch 1/100
389/389 [==============================] - 59s 151ms/step - loss: 0.0875 - dice_coef_0: 0.9857 - dice_coef_1: 0.3360 - dice_coef_2: 0.3899 - val_loss: 0.0610 - val_dice_coef_0: 0.9851 - val_dice_coef_1: 0.2381 - val_dice_coef_2: 0.2478
Epoch 2/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0206 - dice_coef_0: 0.9976 - dice_coef_1: 0.7865 - dice_coef_2: 0.7842 - val_loss: 0.0351 - val_dice_coef_0: 0.9934 - val_dice_coef_1: 0.2718 - val_dice_coef_2: 0.3630
Epoch 3/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0164 - dice_coef_0: 0.9983 - dice_coef_1: 0.8267 - dice_coef_2: 0.8114 - val_loss: 0.0342 - val_dice_coef_0: 0.9945 - val_dice_coef_1: 0.4542 - val_dice_coef_2: 0.6111
Epoch 4/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0130 - dice_coef_0: 0.9987 - dice_coef_1: 0.8262 - dice_coef_2: 0.8057 - val_loss: 0.0174 - val_dice_coef_0: 0.9985 - val_dice_coef_1: 0.9312 - val_dice_coef_2: 0.9463
Epoch 5/

Epoch 34/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0054 - dice_coef_0: 0.9994 - dice_coef_1: 0.9059 - dice_coef_2: 0.9379 - val_loss: 0.0173 - val_dice_coef_0: 0.9983 - val_dice_coef_1: 0.7652 - val_dice_coef_2: 0.9388
Epoch 35/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0051 - dice_coef_0: 0.9994 - dice_coef_1: 0.9253 - dice_coef_2: 0.9348 - val_loss: 0.0226 - val_dice_coef_0: 0.9978 - val_dice_coef_1: 0.7054 - val_dice_coef_2: 0.9280
Epoch 36/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0055 - dice_coef_0: 0.9994 - dice_coef_1: 0.9135 - dice_coef_2: 0.9278 - val_loss: 0.0220 - val_dice_coef_0: 0.9981 - val_dice_coef_1: 0.7848 - val_dice_coef_2: 0.9341
Epoch 37/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0053 - dice_coef_0: 0.9994 - dice_coef_1: 0.9149 - dice_coef_2: 0.9384 - val_loss: 0.0197 - val_dice_coef_0: 0.9982 - val_dice_coef_1: 0.7207 - val_dice_coef_2: 0.9215
Epoc

Epoch 67/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0041 - dice_coef_0: 0.9995 - dice_coef_1: 0.9265 - dice_coef_2: 0.9523 - val_loss: 0.0508 - val_dice_coef_0: 0.9966 - val_dice_coef_1: 0.6695 - val_dice_coef_2: 0.9086
Epoch 68/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0041 - dice_coef_0: 0.9995 - dice_coef_1: 0.9352 - dice_coef_2: 0.9564 - val_loss: 0.0293 - val_dice_coef_0: 0.9979 - val_dice_coef_1: 0.7188 - val_dice_coef_2: 0.9148
Epoch 69/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0057 - dice_coef_0: 0.9994 - dice_coef_1: 0.9147 - dice_coef_2: 0.9428 - val_loss: 0.0161 - val_dice_coef_0: 0.9984 - val_dice_coef_1: 0.6650 - val_dice_coef_2: 0.9058
Epoch 70/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0046 - dice_coef_0: 0.9995 - dice_coef_1: 0.9113 - dice_coef_2: 0.9505 - val_loss: 0.0173 - val_dice_coef_0: 0.9985 - val_dice_coef_1: 0.7429 - val_dice_coef_2: 0.9246
Epoc

Epoch 100/100
389/389 [==============================] - 57s 146ms/step - loss: 0.0036 - dice_coef_0: 0.9996 - dice_coef_1: 0.9322 - dice_coef_2: 0.9617 - val_loss: 0.0258 - val_dice_coef_0: 0.9981 - val_dice_coef_1: 0.6384 - val_dice_coef_2: 0.8772
